# Лабораторная 5 – 10 баллов (дедлайн 25.04.2025)
## Вариант 1 (простой) – 5 баллов
- С помощью библиотеки Optuna настройте гиперпараметры сверточной нейронной сети (например, число слоев, размер фильтра и тд, оптимизатор — не гиперпараметр).
- Гиперпараметры определите сами.
- Гиперпараметров должно быть не менее 5.
- Обучение проводить на датасете CIFAR-10.


## Шаг №0 - Импорт библиотек

In [ ]:
import tensorflow as tf
import numpy as np
import optuna
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os

## Шаг №1 - Проверка доступности GPU и настройка

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print(f"Найдено {len(physical_devices)} GPU:")
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print(f" - {device.name} с динамическим выделением памяти")
else:
    print("GPU не найдена, используется CPU")

## Шаг №2 - Загрузка и предобработка данных

In [ ]:
def load_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    return x_train, y_train, x_test, y_test

## Шаг №3 - Построение модели с гиперпараметрами
## Гиперпараметры:
- Количество сверточных блоков (от 2 до 4)
- Начальное количество фильтров (от 32 до 96)
- Размер ядра свертки (от 3 до 5)
- Скорость обучения (от 1e-4 до 1e-2)
- Коэффициент отсева (Dropout) (от 0.2 до 0.5)
- Использовать ли BatchNormalization
- Количество нейронов в полносвязном слое (от 128 до 512)
- Размер батча (от 32 до 128)

In [ ]:
def create_model(trial):
    n_conv_blocks = trial.suggest_int('n_conv_blocks', 2, 4)

    initial_filters = trial.suggest_int('initial_filters', 32, 96, step=16)

    kernel_size = trial.suggest_int('kernel_size', 3, 5, step=2)

    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)

    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5, step=0.1)

    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])

    model = Sequential()

    for i in range(n_conv_blocks):
        if i == 0:
            model.add(Conv2D(initial_filters * (2 ** i), kernel_size=kernel_size, padding='same',
                             activation='relu', input_shape=(32, 32, 3)))
        else:
            model.add(Conv2D(initial_filters * (2 ** i), kernel_size=kernel_size, padding='same',
                             activation='relu'))

        if use_batch_norm:
            model.add(BatchNormalization())

        model.add(Conv2D(initial_filters * (2 ** i), kernel_size=kernel_size, padding='same',
                         activation='relu'))

        if use_batch_norm:
            model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Dropout(dropout_rate))

    model.add(Flatten())

    dense_units = trial.suggest_int('dense_units', 128, 512, step=64)

    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))

    model.add(Dense(10, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

## Шаг №4 - Определение функции для обучения и оценки модели

In [ ]:
def objective(trial):
    x_train, y_train, x_test, y_test = load_data()

    model = create_model(trial)

    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

    history = model.fit(
        x_train, y_train,
        validation_split=0.1,  # 10% данных для валидации
        epochs=30,  # Максимальное количество эпох
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=1
    )

    _, accuracy = model.evaluate(x_test, y_test, verbose=0)

    return accuracy

## Шаг №5 - Запуск

In [ ]:
def main():
    print("Начинаем оптимизацию гиперпараметров с Optuna...")

    # Создаем исследование Optuna с хранилищем SQLite для параллельного запуска
    study = optuna.create_study(
        direction='maximize',
        study_name='cnn_cifar10_optimization',
        storage='sqlite:///optuna_study.db',
        load_if_exists=True
    )

    study.optimize(objective, n_trials=20, n_jobs=5)

    print("Лучшие гиперпараметры:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")

    print(f"Лучшая достигнутая точность: {study.best_value:.4f}")

    try:
        fig = optuna.visualization.plot_param_importances(study)
        fig.show()

        fig = optuna.visualization.plot_optimization_history(study)
        fig.show()

        fig = optuna.visualization.plot_contour(study, params=['learning_rate', 'dropout_rate'])
        fig.show()
    except:
        print("Не удалось создать визуализацию. Возможно, нужно установить plotly.")

    print("Обучаем лучшую модель...")
    x_train, y_train, _, _ = load_data()
    best_model = create_model(optuna.trial.FixedTrial(study.best_params))
    best_model.fit(x_train, y_train, epochs=30, batch_size=study.best_params['batch_size'])

    best_model.save('best_cnn_cifar10_model.h5')
    print("Лучшая модель сохранена как 'best_cnn_cifar10_model.h5'")


if __name__ == "__main__":
    main()

# Выводы:
## Описание эксперимента
Для проведения эксперимента использовалась библиотека Optuna, которая позволяет автоматизировать процесс подбора гиперпараметров модели. Всего было проведено 30 испытаний с различными комбинациями гиперпараметров.
##Оптимизируемые гиперпараметры:

- Количество сверточных блоков (n_conv_blocks): от 1 до 3
- Начальное количество фильтров (initial_filters): от 32 до 64
- Размер ядра свертки (kernel_size): от 3 до 5
- Коэффициент отсева, Dropout (dropout_rate): от 0.2 до 0.5
- Использование пакетной нормализации (use_batch_norm): True/False
- Количество нейронов в полносвязном слое (dense_units): от 128 до 512
- Скорость обучения (learning_rate): от 1e-4 до 1e-2
- Размер батча (batch_size): 64, 128 или 256

## Результаты
```
Оптимизация завершена!
Общее время выполнения: 88.78 минут

Лучшие гиперпараметры:
  n_conv_blocks: 3
  initial_filters: 64
  kernel_size: 3
  dropout_rate: 0.4
  use_batch_norm: True
  dense_units: 384
  learning_rate: 0.00037519342722137103
  batch_size: 64

Лучшая достигнутая точность: 0.8497

Обучаем лучшую модель на полном наборе данных...
...
Epoch 25/25
704/704 [==============================] - 10s 15ms/step - loss: 0.2205 - accuracy: 0.9235 - val_loss: 0.4853 - val_accuracy: 0.8620

Лучшая модель сохранена в 'prev/best_cnn_cifar10_model.h5'

Важность параметров:
  learning_rate: 0.7477
  n_conv_blocks: 0.1149
  batch_size: 0.0491
  dense_units: 0.0291
  use_batch_norm: 0.0281
  initial_filters: 0.0161
  dropout_rate: 0.0125
  kernel_size: 0.0024

График важности параметров сохранен в 'importance.png'
```

## График важности параметров
![Важность_гиперпараметров](importance.jpg)
## Выводы
1. Наиболее критическим гиперпараметром для данной задачи является скорость обучения (learning rate), что согласуется с общепринятыми представлениями в глубоком обучении.
2. Глубокая архитектура с тремя сверточными блоками превзошла более мелкие модели, что указывает на необходимость достаточной сложности модели для эффективного извлечения признаков из изображений CIFAR-10.
3. Применение техник регуляризации, таких как Dropout и BatchNormalization, значительно улучшило производительность модели.
4. Оптимальный баланс между размером модели и ее производительностью был достигнут при использовании архитектуры средней сложности.